<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

## _*Hamiltonian Parameters*_ 


***
### Contributors
David McKay

## Introduction

The basic architecture of a quantum device for superconducting transmon qubits [[1](#ref1)] is that of several qubits coupled together by resonant bus structures. This can be described by the Hamiltonian 
$$H = \sum_{i=0}^{N_{\text{qubits}}} \left(\omega_i \hat{n}_{i} + \alpha_i \frac{\hat{n}_i}{2} (\hat{n}_i-1) \right) + \sum_{j=0}^{N_{\text{bus}}} \omega_{\text{bus},i} \hat{n}_{\text{bus},j}+ \sum_{i,j} g_{ij} \left(\hat{a}_i \hat{b}_j^{\dagger} + \hat{a}_i^{\dagger} \hat{b}_j\right) $$

where $\omega_i$ is the $0\rightarrow 1$ frequency of qubit $i$, $\alpha_i$ is the anharmonicity of the transmon qubit $i$, $\omega_{\text{bus},j}$ is the frequency of bus $j$ and $g_{ij}$ is the coupling of qubit $i$ to bus $j$. In a planar architure there will only be a few non-zero values of $g_{ij}$. 

**Contents**

[Dressed Basis](#sect1)

[ZZ](#sect2)

[Dispersive Shifts](#sect3)


### References

[1]<a id="ref1"></a> Jens Koch, Terri M. Yu, Jay Gambetta, A. A. Houck, D. I. Schuster, J. Majer, Alexandre Blais, M. H. Devoret, S. M. Girvin and R. J. Schoelkopf. Charge insensitive qubit design derived from the Cooper pair box. https://arxiv.org/abs/cond-mat/0703002

[2]<a id="ref2"></a> Jerry Chow. Quantum Information Processing with Superconducting Qubits. https://rsl.yale.edu/sites/default/files/files/RSL_Theses/jmcthesis.pdf

Code imports and basis functions
==============

In [257]:
import numpy as np 

# qubit frequency (fq) dressed by resonance fres
# interacting with g
def qubit_dressed(fq, fres, g):
    return fq + g**2/(fq-fres)

# exchange interaction between qubits coupled to a 
# common bus
def J(fq1, fq2, fbus, g1, g2):
    d1 = (fq1-fbus)
    d2 = (fq2-fbus)
    return g1*g2*(d1+d2)/2/d1/d2

# state dependent cavity shift
def Chi(fq1, fbus, g, alpha):
    d1 = (fq1-fbus)
    d2 = (fq1+alpha-fbus)
    return g**2*alpha/d1/d2

# build an operator from a list
# using kronecker product
def multiqop(curop=None, oplist=None):
    if curop is None or len(curop)==0:
        return multiqop(oplist[0],oplist[1:])
    if oplist is None or len(oplist)==0:
        return curop
    
    return multiqop(np.kron(curop,oplist[0]),oplist[1:])

def nop(n=1):
    return np.diag(np.arange(n+1,dtype=complex))

def aop(n=1):
    aop_tmp = np.zeros([n+1,n+1],dtype=complex)
    for i in range(n):
        aop_tmp[i,i+1] = np.sqrt(i+1)
    return aop_tmp

def iop(n=1):
    return np.eye(n+1, dtype=complex)

<a id='sect1'></a>

# Dressed Basis

The first assumption made in these systems is that the bus frequency and qubit frequencies are far apart and that the bus is not thermally occupied. Under these assumptions we can rewrite the Hamiltonian in a basis with the bus degrees of freedom removed and an effective coupling between qubits (rewriting without the anharmoncity for simplicity),
$$H = \sum_{i=0}^{N_{\text{qubits}}} \gamma_i \hat{n}_{i} +  \sum_{i,j<i} J_{ij} \left(\hat{a}_i \hat{a}_j^{\dagger} + \hat{a}_i^{\dagger} \hat{a}_j\right) $$

where
$$\gamma_i = \left[\omega_i+\sum_{j}\frac{g_{ij}^2}{\omega_i-\omega_{\text{bus},j}}\right] $$

Noting that the operator $\hat{n}_i, \hat{a}_i$ changed as well. Assuming a pair of qubits are coupled through only a single bus, the exchange term is expressed as ,
$$ J_{ij}=\frac{g_{ik} g_{jk} (\omega_i+\omega_j-2\omega_{\text{bus},k})}{2(\omega_i-\omega_{\text{bus},k})(\omega_j-\omega_{\text{bus},k})} $$

We can further diagonalize,

$$H = \sum_{i=0}^{N_{\text{qubits}}} \lambda_i \hat{\tilde{n}}_{i} $$

where 

$$\lambda_i = \gamma_i + \frac{J^2}{\gamma_i-\gamma_j}$$ 

and so eventual qubits (the two-level systems in the final diagonalized Hamiltonian) are not the same as the bare qubits in our "lab" Hamiltonian. 

**Code**

The code below numerically constructs the Hamiltonian for 2 qubits (infinite anharmonicity) coupled to a single bus. 

In [159]:
#Hamiltonian with n qubits coupled to a single bus
def Hq(fq, fbus, g, alpha=None, qlevels=1, buslevels=1):
    
    oplist_id = [iop(qlevels) for e in range(len(fq))]
    oplist_id.append(iop(buslevels))
    
    if alpha is None:
        alpha = [0 for e in range(len(fq))]
    
    H = 0*multiqop(oplist=oplist_id)
    
    for qind, fqi in enumerate(fq):
        oplist_i = oplist_id.copy()
        oplist_i[qind] = nop(qlevels) 
        
        #add qubit frequency
        nqi_fullop = multiqop(oplist=oplist_i)
        H += fqi*nqi_fullop
        
        #add anharmonicity
        H += 0.5*alpha[qind]*nqi_fullop*(nqi_fullop-1)
        
        oplist_g0 = oplist_id.copy()
        oplist_g1 = oplist_id.copy()
        oplist_g0[qind] = aop(qlevels) 
        oplist_g1[qind] = aop(qlevels).transpose() 
        oplist_g0[-1] = aop(buslevels).transpose() 
        oplist_g1[-1] = aop(buslevels)
        
        #add bus interaction
        H += g[qind]*(multiqop(oplist=oplist_g0)+multiqop(oplist=oplist_g1))
        
    #add bus
    oplist_i = oplist_id.copy()
    oplist_i[-1] = nop(buslevels) 
    H += fbus*multiqop(oplist=oplist_i)
    return H

In [242]:

# Hamiltonian parameters in MHz
g = [70., 70.]
fq = [5000., 5050]
fbus = 6400.

#Hamiltonian with 1 qubit coupled to the bus
H1 = Hq(fq, fbus, [g[0], 0], qlevels=1, buslevels=1)

print("Consider a single qubit coupled to the bus")
print("Shift in the qubit frequency from coupling to the bus: %f MHz"%
      (np.real(sorted(np.linalg.eig(H1)[0])[1])-fq[0]))
print("Shift in the qubit frequency from equation: %f MHz"%(qubit_dressed(fq[0], fbus, g[0])-fq[0]))
print("\n")


#Hamiltonian with 2 qubits coupled to the bus
H1 = Hq(fq, fbus, g, qlevels=1, buslevels=1)
Jcalc = J(fq[0], fq[1], fbus, g[0], g[1])
print("Add Q1:")
print("Exchange Coupling: %f MHz"%(np.abs(Jcalc)))
print("Shift in the Q0 frequency from Q1 (on top of the bus shift): %f MHz"%
      (np.real(sorted(np.linalg.eig(H1)[0])[1])-qubit_dressed(fq[0], fbus, g[0])))
print("Shift in the Q0 frequency from equation: %f MHz"%
      (qubit_dressed(qubit_dressed(fq[0], fbus, g[0]), qubit_dressed(fq[1], fbus, g[1]), Jcalc)-
                                                        qubit_dressed(fq[0], fbus, g[0])))

Consider a single qubit coupled to the bus
Shift in the qubit frequency from coupling to the bus: -3.491293 MHz
Shift in the qubit frequency from equation: -3.500000 MHz


Add Q1:
Exchange Coupling: 3.564815 MHz
Shift in the Q0 frequency from Q1 (on top of the bus shift): -0.233214 MHz
Shift in the Q0 frequency from equation: -0.254819 MHz


<a id='sect2'></a>

# ZZ

One of the consequences of the coupling is that there is a small ``ZZ`` term left over after the diagonalization. So 

$$H = \sum_{i=0}^{N_{\text{qubits}}} \lambda_i \hat{\tilde{n}}_{i} $$

is more realistically

$$H = \sum_{i=0}^{N_{\text{qubits}}} \lambda_i \hat{\tilde{n}}_{i} + \sum_{i,j<i} \xi_{ij} |ij\rangle \langle ij|$$

where 

$$|ij\rangle = \hat{a}_i^{\dagger} \hat{a}_j^{\dagger} |0\rangle^N $$

The ``ZZ`` is typically written in this way because the frequency of each qubit is empirically measured when the other qubits are in the ground state and so

$$\xi_{ij} = \omega_{|ij\rangle} - \omega_i - \omega_j $$

Below we numerically compute ZZ

In [244]:
# Hamiltonian parameters in MHz
g = [70., 75.]
alpha = [-330, -330]
fq = [5000., 5050.]
fbus = 6400.

#Hamiltonian with 1 qubit coupled to the bus
H1 = Hq(fq, fbus, g, alpha=alpha, qlevels=2, buslevels=3)

eigvals = np.real(np.linalg.eig(H1)[0])

# need to be careful here because 
# of how to track from the bare states
# this code works ok except near resonances
inds = [0,0,0]
bare_e = [fq[0], fq[1], fq[0]+fq[1]]
closeness = [1000,1000,1000]
for eind, eigval in enumerate(eigvals):
    for i in range(3):
        if np.abs(eigval-bare_e[i])<closeness[i]:
            closeness[i] = np.abs(eigval-bare_e[i])
            inds[i] = eind
                       
xi = eigvals[inds[2]]-eigvals[inds[1]]-eigvals[inds[0]]

print("ZZ: %f kHz"%(xi*1000))

ZZ: 137.097636 kHz


<a id='sect3'></a>

# Dispersive Shifts

When the qubit is coupled to a bus there is also a dispersive shift of the bus frequency depending on the state of the qubit. For coupling buses this is neglected (there should be no photons in the bus), however, this concept is the basis for qubit measurement with a separate set of readout buses. 

$$H = \omega \hat{n} + \alpha \frac{\hat{n}}{2} (\hat{n}-1)  + \omega_{\text{bus}} \hat{n}_{\text{bus}}+ g \left(\hat{a} \hat{b}^{\dagger} + \hat{a}^{\dagger} \hat{b}\right) $$

is 

$$H = \tilde{\omega} \hat{n} + \tilde{\omega}_{\text{bus}} \hat{n}_{\text{bus}}+ 2\chi \hat{n} \hat{n}_{\text{bus}} $$

where 

$$\chi = \frac{g^2}{\omega-\omega_{\text{bus}}} \frac{\alpha}{\omega+\alpha-\omega_{\text{bus}}} $$

We can look at this numerically as the difference of the cavity frequencies with the qubit in $|0\rangle$ or $|1\rangle$

In [264]:
# Hamiltonian parameters in MHz
g = [50.]
alpha = [-330]
fq = [5000.]
fbus = 6700.

#Hamiltonian with 1 qubit coupled to the bus
H1 = Hq(fq, fbus, g, alpha=alpha, qlevels=2, buslevels=3)

eigvals = np.real(np.linalg.eig(H1)[0])

# need to be careful here because 
# of how to track from the bare states
# this code works ok except near resonances
inds = [0,0,0]
bare_e = [fq[0], fbus, fq[0]+fbus]
closeness = [1000,1000,1000]
for eind, eigval in enumerate(eigvals):
    for i in range(3):
        if np.abs(eigval-bare_e[i])<closeness[i]:
            closeness[i] = np.abs(eigval-bare_e[i])
            inds[i] = eind
                       
chi = (eigvals[inds[2]]-eigvals[inds[0]]-eigvals[inds[1]])/2

print("Chi: %f kHz"%(chi*1000))
print("Chi (Formula): %f kHz"%(Chi(fq[0],fbus,g[0],alpha[0])*1000))

Chi: -238.359594 kHz
Chi (Formula): -239.061142 kHz
